In [1]:
import duckdb
import pandas as pd
import numpy as np
from src.model.utils import train_model
from src.preprocess.etl import get_dataframe
from src.model.inference import predict_month
from src.constants import PATH_DATABASE, MONTHS_BASELINE, MONTHS_INFERENCE, PARAMS


import warnings

warnings.filterwarnings("ignore")

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
con = duckdb.connect(database=PATH_DATABASE, read_only=True)

In [4]:
all_months = MONTHS_BASELINE + MONTHS_INFERENCE
all_months = [str(month) for month in all_months]
where_clause = ", ".join(all_months)
where_clause

'202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106, 202107, 202108, 202109'

In [5]:
df = con.sql(
    f"""
    SELECT
        *
    FROM competencia_03
    WHERE foto_mes IN ({where_clause})
    """
).to_df()

In [6]:
df.head()

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria_202104,clase_ternaria_202105,clase_ternaria_202106,clase_ternaria_202107,clase_ternaria_202108,clase_ternaria_202109
0,142625636,202010,1,0,0,47,47,252.69,-9620.93,848.93,...,122798.76,15.0,0.0,35588.82,CONTINUA,CONTINUA,CONTINUA,CONTINUA,CONTINUA,CONTINUA
1,142625782,202010,1,0,0,39,47,414.63,9078.79,703.87,...,12469.69,3.0,0.0,1642.20,CONTINUA,CONTINUA,CONTINUA,CONTINUA,CONTINUA,CONTINUA
2,142628454,202010,1,0,0,30,47,-203.57,-6326.79,215.09,...,77609.86,17.0,0.0,48386.25,CONTINUA,CONTINUA,CONTINUA,CONTINUA,CONTINUA,CONTINUA
3,142628779,202010,1,0,0,38,47,2520.37,34297.42,460.78,...,1299.29,2.0,0.0,7131.84,CONTINUA,CONTINUA,CONTINUA,CONTINUA,CONTINUA,CONTINUA
4,142631486,202010,1,0,0,33,47,-208.88,-11204.17,-503.20,...,9100.74,17.0,0.0,2463.30,CONTINUA,CONTINUA,CONTINUA,CONTINUA,CONTINUA,CONTINUA


In [7]:
np.sort(df["foto_mes"].unique())

array([202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102,
       202103, 202104, 202105, 202106, 202107, 202108, 202109])

In [8]:
drop_cols = [f"clase_ternaria_{i}" for i in MONTHS_INFERENCE]
drop_cols.append("clase_binaria")
drop_cols

['clase_ternaria_202104',
 'clase_ternaria_202105',
 'clase_ternaria_202106',
 'clase_ternaria_202107',
 'clase_ternaria_202108',
 'clase_ternaria_202109',
 'clase_binaria']

In [9]:
ground_truth = df[["numero_de_cliente","foto_mes", "clase_ternaria_202109"]]
ground_truth = ground_truth[ground_truth["foto_mes"]<=202107]
ground_truth

,numero_de_cliente,foto_mes,clase_ternaria_202109
0,142625636,202010,CONTINUA
1,142625782,202010,CONTINUA
2,142628454,202010,CONTINUA
3,142628779,202010,CONTINUA
4,142631486,202010,CONTINUA
...,...,...,...
2425602,80867796,202106,CONTINUA
2425603,80868026,202106,CONTINUA
2425604,80868240,202106,CONTINUA
2425605,80869734,202106,CONTINUA


In [10]:
ground_truth["ganancia"] = ground_truth["clase_ternaria_202109"].map({"BAJA+2":270000, "BAJA+1":-7000, "CONTINUA":-7000})
ground_truth.head()

,numero_de_cliente,foto_mes,clase_ternaria_202109,ganancia
0,142625636,202010,CONTINUA,-7000
1,142625782,202010,CONTINUA,-7000
2,142628454,202010,CONTINUA,-7000
3,142628779,202010,CONTINUA,-7000
4,142631486,202010,CONTINUA,-7000


In [11]:
training_months = sorted(MONTHS_BASELINE)
training_months = training_months[0:-1]

In [12]:
df["clase_binaria"] = df[f"clase_ternaria_202104"].map({"BAJA+2":1, "BAJA+1":1, "CONTINUA":0})

df_train = df[df["foto_mes"].isin(training_months)].copy()

print("Training months:", df_train["foto_mes"].unique())

df_train = df_train.reset_index(drop=True)

X_train = df_train.drop(columns=drop_cols, axis=1).copy()
y_train_binaria = df_train["clase_binaria"].copy()

for month in MONTHS_INFERENCE:
    print("Pred month:", month)
    tags_monthly = {
        'stage': 'baseline',
        'last_month': month,
    }
    df_test = df[df["foto_mes"].isin([month])].copy()
    df_test = df_test.reset_index(drop=True)
    X_test = df_test.drop(columns=drop_cols, axis=1).copy()
    y_real_ternaria = df_test["clase_ternaria_202109"].copy().map({"BAJA+2":1, "BAJA+1":0, "CONTINUA":0})
    y_real_binaria = df_test["clase_ternaria_202109"].copy().map({"BAJA+2":1, "BAJA+1":1, "CONTINUA":0})
    if month < 202108:
        model = train_model(X_train, y_train_binaria, X_test, y_real_ternaria, y_real_binaria, "baseline", PARAMS, tags_monthly)
    else:
        model = train_model(X_train, y_train_binaria, None, None, None, "baseline", PARAMS, tags_monthly)
    
    preds = X_test[["numero_de_cliente", "foto_mes"]].copy()
    preds["prediction"] = predict_month(model, X_test)
    preds = preds.merge(ground_truth, on=["numero_de_cliente", "foto_mes"])
    
    preds = preds.sort_values(["prediction"], ascending=False)
    
    gan_mes = preds["prediction"] * preds["ganancia"]
    print("Ganancia", month, gan_mes.sum())
    
    preds.to_csv(f"~/buckets/b1/exp_colab/datasets/processed/predictions/baseline/preds_{month}.csv", index=False)

Training months: [202010 202011 202101 202102 202012 202007 202009 202008]
Pred month: 202104


2023/11/15 21:50:21 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during lightgbm autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 5f8bb175c477450ea36fa7e9be7deac9. Failed operations: [MlflowException("Duplicate parameter keys have been submitted: [\'num_boost_round\']. Please ensure the request contains only one param value per param key.")]')]
2023/11/15 21:50:25 WARNING mlflow.utils.requirements_utils: Found datatable version (1.1.0a0+build.1695691407.jauriberamirez) contains a local version label (+build.1695691407.jauriberamirez). MLflow logged a pip requirement for this package as 'datatable==1.1.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/11/15 21:50:25 WARNING mlflow.models.model: Logging model met

Ganancia 202104 -19717176.126806542
Pred month: 202105


2023/11/15 21:52:17 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during lightgbm autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 3d23e810b1e44b5f81034d328f5d2dca. Failed operations: [MlflowException("Duplicate parameter keys have been submitted: [\'num_boost_round\']. Please ensure the request contains only one param value per param key.")]')]
2023/11/15 21:52:19 WARNING mlflow.utils.requirements_utils: Found datatable version (1.1.0a0+build.1695691407.jauriberamirez) contains a local version label (+build.1695691407.jauriberamirez). MLflow logged a pip requirement for this package as 'datatable==1.1.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/11/15 21:52:19 WARNING mlflow.models.model: Logging model met

Ganancia 202105 -86448886.31952368
Pred month: 202106


...

2023/11/15 21:58:03 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during lightgbm autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID cbb9e44db88c4498ac35e0a20f2aae87. Failed operations: [MlflowException("Duplicate parameter keys have been submitted: [\'num_boost_round\']. Please ensure the request contains only one param value per param key.")]')]
2023/11/15 21:58:05 WARNING mlflow.utils.requirements_utils: Found datatable version (1.1.0a0+build.1695691407.jauriberamirez) contains a local version label (+build.1695691407.jauriberamirez). MLflow logged a pip requirement for this package as 'datatable==1.1.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/11/15 21:58:05 WARNING mlflow.models.model: Logging model met

Ganancia 202108 0.0
Pred month: 202109


2023/11/15 21:59:37 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during lightgbm autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID d591780178b2408cb92ea64841fbaf7d. Failed operations: [MlflowException("Duplicate parameter keys have been submitted: [\'num_boost_round\']. Please ensure the request contains only one param value per param key.")]')]
2023/11/15 21:59:40 WARNING mlflow.utils.requirements_utils: Found datatable version (1.1.0a0+build.1695691407.jauriberamirez) contains a local version label (+build.1695691407.jauriberamirez). MLflow logged a pip requirement for this package as 'datatable==1.1.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/11/15 21:59:40 WARNING mlflow.models.model: Logging model met

Ganancia 202109 0.0


<Figure size 1050x700 with 0 Axes>